In [1]:
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE

import IPython.display as display
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


import pandas as pd

tf.__version__

'2.4.1'

In [3]:
mainData = pd.read_csv('./data_labels_mainData.csv')
extraData = pd.read_csv('./data_labels_extraData.csv')

FileNotFoundError: [Errno 2] File ./data_labels_mainData.csv does not exist: './data_labels_mainData.csv'

In [4]:
print(mainData.shape)
print(mainData.info())
print(mainData.describe())
mainData.hist()
plt.show()

NameError: name 'mainData' is not defined

print(extraData.shape)
print(extraData.describe())
extraData.hist()
plt.show()

# Images
27x27 RGB 

In [ ]:
INPUT_DIM = (27,27,3)
HIDDEN_LAYER_DIM = 256
OUTPUT_CLASSES = 2

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=INPUT_DIM),
    tf.keras.layers.Dense(HIDDEN_LAYER_DIM, activation='sigmoid'),
    tf.keras.layers.Dense(OUTPUT_CLASSES)
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='SGD',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['categorical_accuracy'])

For test and training data split via person.
- Check if a person has cancer or not, then would the images for that person have cancer?

In [ ]:
from sklearn.model_selection import train_test_split

with pd.option_context('mode.chained_assignment', None):
    trainData, testData = train_test_split(mainData, test_size=0.2, shuffle=True)
    
with pd.option_context('mode.chained_assignment', None):
    trainData, valData  = train_test_split(trainData, test_size=0.25, shuffle=True)

In [ ]:
trainData['Class'] = trainData['isCancerous'].astype('str')
valData['Class'] = valData['isCancerous'].astype('str')
testData['Class'] = testData['isCancerous'].astype('str')

In [ ]:
trainData.head()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')

batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
        dataframe=trainData,
        directory='./patch_images',
        x_col="ImageName",
        y_col="Class",
        target_size=(27, 27),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_dataframe(
        dataframe=valData,
        directory='patch_images',
        x_col="ImageName",
        y_col="Class",
        target_size=(27, 27),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
history = model.fit_generator(train_generator, validation_data = validation_generator, epochs=150, verbose=0)